## Importações

In [1]:
api_key_openai = ""

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path_dir = "/content/drive/MyDrive/UNICAMP/2° semestre/processamento de linguagem natural/trabalho prático/automatic-essay-score-for-enem/bert-model"
path_dir_save = "/content/drive/MyDrive/UNICAMP/2° semestre/processamento de linguagem natural/trabalho prático/automatic-essay-score-for-enem/llm-models"

In [4]:
!pip install openai -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00


In [5]:
import os
import sys
import time
sys.path.append(path_dir)

import numpy as np

from openai import OpenAI

from torch.utils.data import DataLoader, Dataset

import pandas as pd

from build_dataset import Corpus

## Carregando dados de treino, validação e teste

In [6]:
def build_prompt(essay, competencia):
  # TODO colocar isso antes de (redacao para análise):
  # # Tema da Redação para Análise
  # {topic}
  try:
    if competencia == 0:
      # construir prompt para competencia I
      return f'''
      # Contexto
      Você é um corretor de redação do ENEM, exame realizado anualmente no Brasil para avaliar estudantes ao final do ensino médio. A correção da redação é feita com base em cinco competências específicas, cada uma com critérios e pontuações detalhadas. Abaixo, concentre-se na Competência I: Demonstrar domínio da modalidade escrita formal da Língua Portuguesa.

      # Redação para Análise
      {essay}

      # Escala e detalhes para correção na Competência I
      Na Competência I você deve avaliar a capacidade do estudante nos seguintes tópicos contidos em uma escala de 0 a 5, onde 0 indica baixo domínio e 5 indica domínio excelente:
      - 0: Demonstra desconhecimento da modalidade escrita formal da Língua Portuguesa. Detalhes dos elementos que devem ser identificados para classificação da redação: Estrutura sintática inexistente (independentemente da quantidade de desvios).
      - 1: Demonstra domínio precário da modalidade escrita formal da Língua Portuguesa, de forma sistemática, com diversificados e frequentes desvios gramaticais, de escolha de registro e de convenções da escrita. Detalhes dos elementos que devem ser identificados para classificação da redação: Estrutura sintática deficitária com muitos desvios.
      - 2: Demonstra domínio insuficiente da modalidade escrita formal da Língua Portuguesa, com muitos desvios gramaticais, de escolha de registro e de convenções da escrita. Detalhes dos elementos que devem ser identificados para classificação da redação: Estrutura sintática deficitária OU muitos desvios.
      - 3: Demonstra domínio mediano da modalidade escrita formal da Língua Portuguesa e de escolha de registro, com alguns desvios gramaticais e de convenções da escrita. Detalhes dos elementos que devem ser identificados para classificação da redação: Estrutura sintática regular E alguns desvios.
      - 4: Demonstra bom domínio da modalidade escrita formal da Língua Portuguesa e de escolha de registro, com poucos desvios gramaticais e de convenções da escrita. Detalhes dos elementos que devem ser identificados para classificação da redação: Estrutura sintática boa E poucos desvios.
      - 5: Demonstra excelente domínio da modalidade escrita formal da Língua Portuguesa e de escolha de registro. Desvios gramaticais ou de convenções da escrita serão aceitos somente como excepcionalidade e quando não caracterizarem reincidência. Detalhes dos elementos que devem ser identificados para classificação da redação: Estrutura sintática excelente (no máximo, uma falha) E, no máximo, dois desvios.

      # Mapeamento da escala com as notas que devem ser atribuidas:
      - 0: nota 0
      - 1: nota 40
      - 2: nota 80
      - 3: nota 120
      - 4: nota 160
      - 5: nota 200

      # Instruções de Correção para Competência I
      1 - Análise: Examine atentamente a redação e identifique os elementos que indicam o nível de domínio na competência I, seguindo os detalhes informados para correção da competência I.
      2 - Feedback construtivo: Escreva um feedback direcionado ao estudante, apontando erros e explicando como ele pode melhorar. Indique pontos fortes e fracos, sugerindo melhorias específicas.
      3 - Pontuação: Atribua uma nota de 0 a 200 com base no mapeamento entre a escala e a nota informada acima.

      # Formato de Resposta
      Responda no seguinte formato:

      Competência I
      feedback: [Feedback construtivo específico para o domínio da língua portuguesa, destacando erros frequentes e sugestões de melhoria.]
      nota: [Pontuação final para a Competência I]
      '''
    elif competencia == 1:
      # construir prompt para competencia II
      return f'''
      # Contexto
      Você é um corretor de redação do ENEM, exame realizado anualmente no Brasil para avaliar estudantes ao final do ensino médio. A correção da redação é feita com base em cinco competências específicas, cada uma com critérios e pontuações detalhadas. Abaixo, concentre-se na Competência II: Compreender a proposta de redação e aplicar conceitos das várias áreas de conhecimento para desenvolver o tema, dentro dos limites estruturais do texto dissertativo-argumentativo em prosa.

      # Redação para Análise
      {essay}

      # Escala e detalhes para correção na Competência II
      Na Competência II você deve avaliar a capacidade do estudante nos seguintes tópicos contidos em uma escala de 1 a 5, onde 1 indica baixo domínio e 5 indica domínio excelente:
      - 1: Apresenta o assunto, tangenciando o tema, ou demonstra domínio precário do texto dissertativo-argumentativo, com traços constantes de outros tipos textuais. Detalhes dos elementos que devem ser identificados para classificação da redação: Tangência ao tema OU Texto composto por aglomerado de palavras OU Traços constantes de outros tipos textuais.
      - 2: Desenvolve o tema recorrendo à cópia de trechos dos textos motivadores ou apresenta domínio insuficiente do texto dissertativo-argumentativo, não atendendo à estrutura com proposição, argumentação e conclusão. Detalhes dos elementos que devem ser identificados para classificação da redação: Abordagem completa do tema E 3 partes do texto (2 delas embrionárias) OU Conclusão finalizada por frase incompleta. Textos que apresentam muitos trechos de cópias dos textos motivadores não devem ultrapassar esse nível.
      - 3: Desenvolve o tema por meio de argumentação previsível e apresenta domínio mediano do texto dissertativo-argumentativo, com proposição, argumentação e conclusão. Detalhes dos elementos que devem ser identificados para classificação da redação: Abordagem completa do tema E 3 partes do texto (1 parte pode ser embrionária) E Repertório baseado nos textos motivadores E/OU Repertório não legitimado E/OU Repertório legitimado, MAS não pertinente ao tema.
      - 4: Desenvolve o tema por meio de argumentação consistente e apresenta bom domínio do texto dissertativo-argumentativo, com proposição, argumentação e conclusão. Detalhes dos elementos que devem ser identificados para classificação da redação: Abordagem completa do tema E 3 partes do texto (nenhuma delas embrionária) E Repertório legitimado E pertinente ao tema, MAS com uso improdutivo.
      - 5: Desenvolve o tema por meio de argumentação consistente, a partir de um repertório sociocultural produtivo, e apresenta excelente domínio do texto dissertativo-argumentativo. Detalhes dos elementos que devem ser identificados para classificação da redação: Abordagem completa do tema E 3 partes do texto (nenhuma delas embrionária) E Repertório legitimado E pertinente ao tema, COM uso produtivo.

      # Mapeamento da escala com as notas que devem ser atribuídas:
      - 1: nota 40
      - 2: nota 80
      - 3: nota 120
      - 4: nota 160
      - 5: nota 200

      # Instruções de Correção para Competência II
      1 - Análise: Examine atentamente a redação e identifique os elementos que indicam o nível de domínio na competência II, seguindo os detalhes informados para correção da competência II.
      2 - Feedback construtivo: Escreva um feedback direcionado ao estudante, abordando erros e explicando como ele pode melhorar. Destaque pontos fortes e fracos, com sugestões específicas de melhoria.
      3 - Pontuação: Atribua uma nota de 0 a 200 com base no mapeamento entre a escala e a nota informada acima.

      # Formato de Resposta
      Responda no seguinte formato:

      Competência II
      feedback: [Feedback construtivo específico para o desenvolvimento do tema e estrutura dissertativo-argumentativa, destacando erros frequentes e sugestões de melhoria.]
      nota: [Pontuação final para a Competência II]
      '''
    elif competencia == 2:
      # construir prompt para competencia III
      return f'''
      # Contexto
      Você é um corretor de redação do ENEM, exame realizado anualmente no Brasil para avaliar estudantes ao final do ensino médio. A correção da redação é feita com base em cinco competências específicas, cada uma com critérios e pontuações detalhadas. Abaixo, concentre-se na Competência III: Selecionar, relacionar, organizar e interpretar informações, fatos, opiniões e argumentos em defesa de um ponto de vista.

      # Redação para Análise
      {essay}

      # Escala e detalhes para correção na Competência III
      Na Competência III você deve avaliar a capacidade do estudante nos seguintes tópicos contidos em uma escala de 0 a 5, onde 0 indica pouco domínio e 5 indica excelente domínio:
      - 0: Apresenta informações, fatos e opiniões não relacionados ao tema e sem defesa de um ponto de vista. Detalhes dos elementos que devem ser identificados para classificação da redação: Tangente ao tema e sem direção.
      - 1: Apresenta informações, fatos e opiniões pouco relacionados ao tema ou incoerentes e sem defesa de um ponto de vista. Detalhes dos elementos que devem ser identificados para classificação da redação: Tangente ao tema e com direção OU Abordagem completa do tema e sem direção.
      - 2: Apresenta informações, fatos e opiniões relacionados ao tema, mas desorganizados ou contraditórios e limitados aos argumentos dos textos motivadores, em defesa de um ponto de vista. Detalhes dos elementos que devem ser identificados para classificação da redação: Projeto de texto com muitas falhas E Sem desenvolvimento ou com desenvolvimento de apenas uma informação, fato ou opinião. Textos que apresentam contradição grave não devem ultrapassar este nível.
      - 3: Apresenta informações, fatos e opiniões relacionados ao tema, limitados aos argumentos dos textos motivadores e pouco organizados, em defesa de um ponto de vista. Detalhes dos elementos que devem ser identificados para classificação da redação: Projeto de texto com algumas falhas E Desenvolvimento de algumas informações, fatos e opiniões.
      - 4: Apresenta informações, fatos e opiniões relacionados ao tema, de forma organizada, com indícios de autoria, em defesa de um ponto de vista. Detalhes dos elementos que devem ser identificados para classificação da redação: Projeto de texto com poucas falhas E Desenvolvimento da maior parte das informações, fatos e opiniões.
      - 5: Apresenta informações, fatos e opiniões relacionados ao tema proposto, de forma consistente e organizada, configurando autoria, em defesa de um ponto de vista. Detalhes dos elementos que devem ser identificados para classificação da redação: Projeto de texto estratégico E Desenvolvimento das informações, fatos e opiniões em todo o texto. Aqui se admitem deslizes pontuais, sejam de projeto e/ou de desenvolvimento.

      # Mapeamento da escala com as notas que devem ser atribuídas:
      - 0: nota 0
      - 1: nota 40
      - 2: nota 80
      - 3: nota 120
      - 4: nota 160
      - 5: nota 200

      # Instruções de Correção para Competência III
      1 - Análise: Examine atentamente a redação e identifique os elementos que indicam o nível de domínio na competência III, seguindo os detalhes informados para correção da competência III.
      2 - Feedback construtivo: Escreva um feedback direcionado ao estudante, apontando erros e explicando como ele pode melhorar. Indique pontos fortes e fracos, sugerindo melhorias específicas.
      3 - Pontuação: Atribua uma nota de 0 a 200 com base no mapeamento entre a escala e a nota informada acima.

      # Formato de Resposta
      Responda no seguinte formato:

      Competência III
      feedback: [Feedback construtivo específico para a seleção e organização de informações e defesa de um ponto de vista, destacando pontos fortes e sugestões de melhoria.]
      nota: [Pontuação final para a Competência III]
      '''
    elif competencia == 3:
      # construir prompt para competencia IV
      return f'''
      # Contexto
      Você é um corretor de redação do ENEM, exame realizado anualmente no Brasil para avaliar estudantes ao final do ensino médio. A correção da redação é feita com base em cinco competências específicas, cada uma com critérios e pontuações detalhadas. Abaixo, concentre-se na Competência IV: Demonstrar conhecimento dos mecanismos linguísticos necessários para a construção da argumentação.

      # Redação para Análise
      {essay}

      # Escala e detalhes para correção na Competência IV
      Na Competência IV você deve avaliar a capacidade do estudante nos seguintes tópicos contidos em uma escala de 0 a 5, onde 0 indica ausência de articulação e 5 indica excelente articulação e diversidade de recursos coesivos:
      - 0: Não articula as informações. Detalhes dos elementos que devem ser identificados para classificação da redação: Palavras e períodos justapostos e desconexos ao longo de todo o texto, o que demonstra ausência de articulação.
      - 1: Articula as partes do texto de forma precária. Detalhes dos elementos que devem ser identificados para classificação da redação: Presença rara de elementos coesivos inter e/ou intraparágrafos E/OU excessivas repetições E/OU excessivas inadequações.
      - 2: Articula as partes do texto de forma insuficiente, com muitas inadequações, e apresenta repertório limitado de recursos coesivos. Detalhes dos elementos que devem ser identificados para classificação da redação: Presença pontual de elementos coesivos inter e/ ou intraparágrafos E/OU muitas repetições E/OU muitas inadequações. Textos em forma de monobloco não devem ultrapassar este nível.
      - 3: Articula as partes do texto de forma mediana, com inadequações, e apresenta repertório pouco diversificado de recursos coesivos. Detalhes dos elementos que devem ser identificados para classificação da redação: Presença regular de elementos coesivos inter E/OU intraparágrafos E/OU algumas repetições E/OU algumas inadequações.
      - 4: Articula as partes do texto, com poucas inadequações, e apresenta repertório diversificado de recursos coesivos. Detalhes dos elementos que devem ser identificados para classificação da redação: Presença constante de elementos coesivos inter (havendo elemento coesivo de tipo “operador argumentativo” entre parágrafos em, pelo menos, 01 momento do texto) e intraparágrafos E/OU poucas repetições E/OU poucas inadequações.
      - 5: Articula bem as partes do texto e apresenta repertório diversificado de recursos coesivos. Detalhes dos elementos que devem ser identificados para classificação da redação: Presença expressiva de elementos coesivos inter** e intraparágrafos (havendo elemento coesivo de tipo "operador argumentativo" entre parágrafos em, pelo menos, 02 momentos do texto e, pelo menos, 01 elemento coesivo de qualquer tipo dentro de todos os parágrafos) E raras ou ausentes repetições E sem inadequação.

      # Roteiro de avaliação da competência IV
      - A: É monobloco, ou seja, só tem um parágrafo? Se sim, a escala seria 0, 1 ou 2?
      - B: Há presença de elementos coesivos intra e interparágrafos ao longo de todo o texto?
      - C: Há repetição de elementos coesivos do tipo operador argumentativo?
      - D: O uso de elementos coesivos é adequado, ou seja, ele contribui para a articulação de argumentos?
      - E: Há características de níveis de nota diferentes? Se sim, avaliar em escala mais baixa0.

      # Mapeamento da escala com as notas que devem ser atribuídas:
      - 0: nota 0
      - 1: nota 40
      - 2: nota 80
      - 3: nota 120
      - 4: nota 160
      - 5: nota 200

      # Instruções de Correção para Competência IV
      1 - Análise: Examine atentamente a redação e identifique os elementos que indicam o nível de articulação textual e o uso de coesão, seguindo os detalhes informados para a correção da competência IV.
      2 - Feedback construtivo: Escreva um feedback direcionado ao estudante, apontando erros e explicando como ele pode melhorar. Indique pontos fortes e fracos, sugerindo melhorias específicas.
      3 - Pontuação: Atribua uma nota de 0 a 200 com base no mapeamento entre a escala e a nota informada acima.

      # Formato de Resposta
      Responda no seguinte formato:

      Competência IV
      feedback: [Feedback construtivo específico para o uso de coesão e articulação dos argumentos, indicando melhorias no uso de recursos coesivos e sugestões para reduzir inadequações ou repetições.]
      nota: [Pontuação final para a Competência IV]
      '''
    elif competencia == 4:
      # construir prompt para competencia V
      return f'''
      # Contexto
      Você é um corretor de redação do ENEM, exame realizado anualmente no Brasil para avaliar estudantes ao final do ensino médio. A correção da redação é feita com base em cinco competências específicas, cada uma com critérios e pontuações detalhadas. Abaixo, concentre-se na Competência V: Elaborar proposta de intervenção para o problema abordado, respeitando os direitos humanos.

      # Redação para Análise
      {essay}

      # Escala e detalhes para correção na Competência V
      Na Competência V você deve avaliar a capacidade do estudante nos seguintes tópicos contidos em uma escala de 0 a 5, onde 0 indica pouco domínio e 5 indica excelente domínio:
      - 0: Não apresenta proposta de intervenção ou apresenta proposta não relacionada ao tema ou ao assunto. Detalhes dos elementos que devem ser identificados para classificação da redação: Ausência de proposta ou cópia integral de proposta OU Proposta de intervenção que desrespeita os direitos humanos OU Proposta de intervenção não relacionada sequer ao assunto.
      - 1: Apresenta proposta de intervenção vaga, precária ou relacionada apenas ao assunto. Detalhes dos elementos que devem ser identificados para classificação da redação: Tangenciamento do tema OU Apenas elemento(s) nulo(s) OU 1 elemento válido.
      - 2: Elabora, de forma insuficiente, proposta de intervenção relacionada ao tema, ou não articulada com a discussão desenvolvida no texto. Detalhes dos elementos que devem ser identificados para classificação da redação: 2 elementos válidos. Estruturas condicionais com 2 ou mais elementos válidos não devem ultrapassar este nível.
      - 3: Elabora, de forma mediana, proposta de intervenção relacionada ao tema e articulada à discussão desenvolvida no texto. Detalhes dos elementos que devem ser identificados para classificação da redação: 3 elementos válidos.
      - 4: Elabora bem proposta de intervenção relacionada ao tema e articulada à discussão desenvolvida no texto. Detalhes dos elementos que devem ser identificados para classificação da redação: 4 elementos válidos.
      - 5: Elabora muito bem proposta de intervenção, detalhada, relacionada ao tema e articulada à discussão desenvolvida no texto. Detalhes dos elementos que devem ser identificados para classificação da redação: 5 elementos válidos.

      # Mapeamento da escala com as notas que devem ser atribuídas:
      - 0: nota 0
      - 1: nota 40
      - 2: nota 80
      - 3: nota 120
      - 4: nota 160
      - 5: nota 200

      # Instruções de Correção para Competência V
      1 - Análise: Examine atentamente a redação e identifique os elementos que indicam o nível de elaboração da proposta de intervenção, respeitando os direitos humanos, conforme os detalhes informados para correção da Competência V.
      2 - Feedback construtivo: Escreva um feedback direcionado ao estudante, apontando erros e explicando como ele pode melhorar. Indique pontos fortes e fracos, sugerindo melhorias específicas.
      3 - Pontuação: Atribua uma nota de 0 a 200 com base no mapeamento entre a escala e a nota informada acima.

      # Formato de Resposta
      Responda no seguinte formato:

      Competência V
      feedback: [Feedback construtivo específico para a seleção e organização de informações e defesa de um ponto de vista, destacando pontos fortes e sugestões de melhoria.]
      nota: [Pontuação final para a Competência V]
      '''
    else:
      raise ValueError("Competência inválida. Deve ser um número entre 0 (competência 1) e 4 (competência 5).")
  except Exception as e:
        return f"Ocorreu um erro: {str(e)}"

In [7]:
class EssayDataset(Dataset):
  def __init__(self, essays, scores):
    self.essays = essays
    self.scores = scores

  def __len__(self):
    return len(self.essays)

  def __getitem__(self, idx):
    redacao = self.essays[idx]
    # competencia = 0 é a nota da C1
    # competencia = 1 é a nota da C2
    # competencia = 2 é a nota da C3
    # competencia = 3 é a nota da C4
    # competencia = 4 é a nota da C5
    scores = self.scores[idx] # array que vai de 0 (competencia 1) até 4 (competencia 5)
    # print(len(scores))

    # class_index = score // 40

    prompts = []
    for i, score in enumerate(scores):
      prompts.append(build_prompt(essay=redacao, competencia=i))

    return {
        'prompts': prompts,
        'scores': scores,
        'essay': redacao
    }

In [8]:
def create_data_loader(essays, scores, batch_size=64, num_workers=4):
    ds = EssayDataset(
        essays=essays,
        scores=scores
    )
    # print(ds.__getitem__(0))
    # ds.__getitem__(0)
    return ds, DataLoader(ds, batch_size=batch_size, shuffle=False, num_workers=num_workers)

In [9]:
train, valid, test = Corpus().read_splits(root_path=f"{path_dir}/extended-corpus/splits")

# train['essay'] = train['essay'].apply(lambda x: ''.join(x))
# valid['essay'] = valid['essay'].apply(lambda x: ''.join(x))
test['essay'] = test['essay'].apply(lambda x: ''.join(x))

# lista_redacoes_train = train['essay'].tolist()
# lista_competencias_train = train[['c1', 'c2', 'c3', 'c4', 'c5']].values.tolist()

# lista_redacoes_valid = valid['essay'].tolist()
# lista_competencias_valid = valid[['c1', 'c2', 'c3', 'c4', 'c5']].values.tolist()

lista_redacoes_test = test['essay'].tolist()
lista_competencias_test = test[['c1', 'c2', 'c3', 'c4', 'c5']].values.tolist()

# train_data_loader = create_data_loader(lista_redacoes_train, lista_competencias_train, 5, 2)
# val_data_loader = create_data_loader(lista_redacoes_valid, lista_competencias_valid, 5, 2)
test_dataset, test_data_loader = create_data_loader(lista_redacoes_test, lista_competencias_test, 5, 2)

In [10]:
len(test_dataset)

987

In [11]:
# testando dataset
for i, data in enumerate(test_dataset):
  print("data: ", i)
  print(data)
  break

data:  0
{'prompts': ['\n      # Contexto\n      Você é um corretor de redação do ENEM, exame realizado anualmente no Brasil para avaliar estudantes ao final do ensino médio. A correção da redação é feita com base em cinco competências específicas, cada uma com critérios e pontuações detalhadas. Abaixo, concentre-se na Competência I: Demonstrar domínio da modalidade escrita formal da Língua Portuguesa.\n\n      # Redação para Análise\n      A Amazônia é um bioma importantíssimo para o Brasil e para outros países.Os motivos para o desmatamento da Amazônia são diversos, os mais importantes fatores que tm provocado à derrubada florestal são: o desmate para a pecuária intensiva, avança da agricultar de grãos e desmate para a especulação fundiária. Esses desmatamentos vem acompanhados de queimadas, pois é uma forma na preparação do solo para fazer s plantações de agricultara extensivas. Existem também incêndios naturais devido elevação das temperaturas.A falta de incentivos ao controle do d

In [12]:
# testando dataloader
for i, data in enumerate(test_data_loader):
  print("data: ", i)
  print(data)
  break

data:  0
{'prompts': [('\n      # Contexto\n      Você é um corretor de redação do ENEM, exame realizado anualmente no Brasil para avaliar estudantes ao final do ensino médio. A correção da redação é feita com base em cinco competências específicas, cada uma com critérios e pontuações detalhadas. Abaixo, concentre-se na Competência I: Demonstrar domínio da modalidade escrita formal da Língua Portuguesa.\n\n      # Redação para Análise\n      A Amazônia é um bioma importantíssimo para o Brasil e para outros países.Os motivos para o desmatamento da Amazônia são diversos, os mais importantes fatores que tm provocado à derrubada florestal são: o desmate para a pecuária intensiva, avança da agricultar de grãos e desmate para a especulação fundiária. Esses desmatamentos vem acompanhados de queimadas, pois é uma forma na preparação do solo para fazer s plantações de agricultara extensivas. Existem também incêndios naturais devido elevação das temperaturas.A falta de incentivos ao controle do 

## Testando gpt-4o-mini
- RODAR SOMENTE PARA TESTES

In [11]:
sample_test = test_dataset[0]
print(sample_test)

{'prompts': ['\n      # Contexto\n      Você é um corretor de redação do ENEM, exame realizado anualmente no Brasil para avaliar estudantes ao final do ensino médio. A correção da redação é feita com base em cinco competências específicas, cada uma com critérios e pontuações detalhadas. Abaixo, concentre-se na Competência I: Demonstrar domínio da modalidade escrita formal da Língua Portuguesa.\n\n      # Redação para Análise\n      A Amazônia é um bioma importantíssimo para o Brasil e para outros países.Os motivos para o desmatamento da Amazônia são diversos, os mais importantes fatores que tm provocado à derrubada florestal são: o desmate para a pecuária intensiva, avança da agricultar de grãos e desmate para a especulação fundiária. Esses desmatamentos vem acompanhados de queimadas, pois é uma forma na preparação do solo para fazer s plantações de agricultara extensivas. Existem também incêndios naturais devido elevação das temperaturas.A falta de incentivos ao controle do desmatamen

In [24]:
print(sample_test['prompts'][4])
print(sample_test['scores'][4])


      # Contexto
      Você é um corretor de redação do ENEM, exame realizado anualmente no Brasil para avaliar estudantes ao final do ensino médio. A correção da redação é feita com base em cinco competências específicas, cada uma com critérios e pontuações detalhadas. Abaixo, concentre-se na Competência V: Elaborar proposta de intervenção para o problema abordado, respeitando os direitos humanos.

      # Redação para Análise
      A Amazônia é um bioma importantíssimo para o Brasil e para outros países.Os motivos para o desmatamento da Amazônia são diversos, os mais importantes fatores que tm provocado à derrubada florestal são: o desmate para a pecuária intensiva, avança da agricultar de grãos e desmate para a especulação fundiária. Esses desmatamentos vem acompanhados de queimadas, pois é uma forma na preparação do solo para fazer s plantações de agricultara extensivas. Existem também incêndios naturais devido elevação das temperaturas.A falta de incentivos ao controle do desmata

In [25]:
client = OpenAI(api_key=api_key_openai)

# TODO: fazer loop aqui no test_data_loader e carregar o prompt e coletar o resultado para pós-processamento
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "Você é um corretor de redação do ENEM e segue as definições de correção das competências do ENEM."},
        {
            "role": "user",
            "content": sample_test['prompts'][4]
        }
    ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Competência V  \nfeedback: A proposta de intervenção apresentada tem alguns elementos válidos, como a necessidade de fiscalização ambiental e a colaboração entre empresas, ONGs e a sociedade para a preservação da Amazônia. No entanto, ela carece de mais detalhamento e articulação com os problemas discutidos anteriormente no texto. Para melhorar, recomendo que elabore a proposta com mais clareza, especificando como a sociedade e as empresas podem efetivamente colaborar, incluindo, por exemplo, educar a população sobre práticas sustentáveis ou criar programas de reflorestamento. Também é importante lembrar da questão dos direitos humanos, assegurando que a intervenção respeite as comunidades locais e suas necessidades. Envolva uma discussão mais aprofundada sobre como garantir que as ações não impactem negativamente as populações que dependem da Amazônia para viver.  \nnota: 120', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=No

## Treinamento
- RODAR SOMENTE SE NÃO TIVER PREDICTIONS.NPY

In [13]:
client = OpenAI(api_key=api_key_openai)

predictions = []

MAX_REQUESTS_PER_MINUTE = 250 # maximo é 500
MAX_TOKENS_PER_MINUTE = 100_000 # máximo é 200_000

# Contador de requisições e tokens processados no minuto atual
request_count = 0
token_count = 0

for idx, data in enumerate(test_dataset):
  essay = data['essay']

  for i, score in enumerate(data['scores']):
    prompt = data['prompts'][i]

    # Verifica se o limite de requisições ou tokens por minuto foi atingido
    if request_count >= MAX_REQUESTS_PER_MINUTE or token_count >= MAX_TOKENS_PER_MINUTE:
      print("Aguardando 30 segundos para evitar exceder os limites...")
      time.sleep(30)  # Aguarda exatamente 30 segundos

      # Reinicia os contadores e o cronômetro para o próximo minuto
      request_count = 0
      token_count = 0

    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Você é um corretor de redação do ENEM e segue as definições de correção das competências do ENEM."},
            {"role": "user", "content": prompt}
        ]
    )

    response_message = completion.choices[0].message

    # Atualiza os contadores
    tokens_used = completion.usage.total_tokens  # ou uma estimativa baseada no prompt e resposta
    print("tokens usados na última requisição: ", tokens_used)
    token_count += tokens_used
    request_count += 1

    # Armazena a predição
    predictions.append((response_message, {'essay': essay, 'score': score}))

Streaming output truncated to the last 5000 lines.
tokens usados na última requisição:  1841
tokens usados na última requisição:  1492
tokens usados na última requisição:  1393
tokens usados na última requisição:  1496
tokens usados na última requisição:  1442
tokens usados na última requisição:  1670
tokens usados na última requisição:  1333
tokens usados na última requisição:  1390
tokens usados na última requisição:  1538
tokens usados na última requisição:  1437
tokens usados na última requisição:  1702
tokens usados na última requisição:  1334
tokens usados na última requisição:  1378
tokens usados na última requisição:  1553
tokens usados na última requisição:  1425
tokens usados na última requisição:  1654
tokens usados na última requisição:  1318
tokens usados na última requisição:  1494
tokens usados na última requisição:  1648
tokens usados na última requisição:  1554
tokens usados na última requisição:  1793
tokens usados na última requisição:  1431
tokens usados na última r

In [14]:
len(predictions)

4935

In [19]:
predictions[:2]

[(ChatCompletionMessage(content='Competência I  \nfeedback: A redação apresenta uma estrutura sintática regular, mas é necessário melhorar alguns aspectos gramaticais e de convenções da escrita. Os principais problemas incluem: erros de concordância, como em "fatores que tm provocado à derrubada" (o correto seria "provocados") e "desmate para a pecuária intensiva" (o correto é "desmatamento"). Além disso, o uso de "avança da agricultar de grãos" deve ser revisto para "avanço da agricultura de grãos". O uso excessivo de vírgulas e a ausência de espaços entre algumas palavras (como "Amazônia é um bioma importantíssimo para o Brasil e para outros países.Os") também prejudicam a fluidez do texto. Para melhorar, recomenda-se revisar as regras de concordância e pontuação, além de uma reestruturação das frases para evitar confusões.  \nnota: 120', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None),
  {'essay': 'A Amazônia é um bioma importantíssimo para o Brasil 

In [15]:
# Salvando como arquivo .npy
np.save(path_dir_save+'/predictions.npy', predictions)

## Calculando F1-Score, Accuracy, QWK e GPT_EVAL

### Carregando testes

In [6]:
loaded_predictions = np.load(path_dir_save+'/predictions.npy', allow_pickle=True)
print(len(loaded_predictions))

4935


In [7]:
loaded_predictions[:2]

array([[ChatCompletionMessage(content='Competência I  \nfeedback: A redação apresenta uma estrutura sintática regular, mas é necessário melhorar alguns aspectos gramaticais e de convenções da escrita. Os principais problemas incluem: erros de concordância, como em "fatores que tm provocado à derrubada" (o correto seria "provocados") e "desmate para a pecuária intensiva" (o correto é "desmatamento"). Além disso, o uso de "avança da agricultar de grãos" deve ser revisto para "avanço da agricultura de grãos". O uso excessivo de vírgulas e a ausência de espaços entre algumas palavras (como "Amazônia é um bioma importantíssimo para o Brasil e para outros países.Os") também prejudicam a fluidez do texto. Para melhorar, recomenda-se revisar as regras de concordância e pontuação, além de uma reestruturação das frases para evitar confusões.  \nnota: 120', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None),
        {'essay': 'A Amazônia é um bioma importantíssimo pa

### Preparando extração

In [8]:
import re

def extrair_notas_e_feedbacks(mensagens):
    # Lista para armazenar as notas e feedbacks extraídos de cada competência
    resultados = {
        'Competência I': [],
        'Competência II': [],
        'Competência III': [],
        'Competência IV': [],
        'Competência V': []
    }

    for item in mensagens:
        conteudo = item[0].content  # Acessa o campo `content` da mensagem do GPT
        # print(conteudo)
        nota_gt = item[1]['score']
        essay = item[1]['essay']

        # Extrair a competência a partir do conteúdo
        competencia_match = re.search(r"Competência\s+([IVXLCDM]+)", conteudo)

        # Verifica se a competência foi encontrada
        if competencia_match:
            competencia = competencia_match.group(0)
        else:
            print(f"Competência não encontrada no conteúdo: {conteudo}")
            continue  # Se a competência não for encontrada, pula para o próximo item

        # Extrair a nota e o feedback
        nota_match = re.search(r"nota: (\d+)", conteudo)
        nota_match_com_asterisco = re.search(r"\*\*nota:\*\* (\d+)", conteudo)
        feedback_match = re.search(r"feedback:\s*(.*)", conteudo)
        feedback_match_com_asterisco = re.search(r"\*\*feedback:\*\* (.*)", conteudo)

        # Armazena a nota e o feedback extraídos
        # Verifica se a nota foi encontrada
        if nota_match:
          nota = int(nota_match.group(1))
        elif nota_match_com_asterisco:
          nota = int(nota_match_com_asterisco.group(1))
        else:
          nota = 0  # Definindo nota padrão como 0
          print(f"Nota não encontrada no conteúdo: {conteudo}")

        # Verifica se o feedback foi encontrado
        if feedback_match:
          feedback = feedback_match.group(1).strip()
        elif feedback_match_com_asterisco:
          feedback = feedback_match_com_asterisco.group(1).strip()
        else:
          feedback = "Feedback não encontrado"
          print(f"Feedback não encontrado no conteúdo: {conteudo}")

        # Adiciona um dicionário com nota e feedback aos resultados
        resultados[competencia].append({
            'nota_predita': nota,
            'nota_real': nota_gt,
            'feedback': feedback,
            'redacao': essay
        })

    return resultados

In [9]:
extrair_notas_e_feedbacks(loaded_predictions[:1])

{'Competência I': [{'nota_predita': 120,
   'nota_real': 120,
   'feedback': 'A redação apresenta uma estrutura sintática regular, mas é necessário melhorar alguns aspectos gramaticais e de convenções da escrita. Os principais problemas incluem: erros de concordância, como em "fatores que tm provocado à derrubada" (o correto seria "provocados") e "desmate para a pecuária intensiva" (o correto é "desmatamento"). Além disso, o uso de "avança da agricultar de grãos" deve ser revisto para "avanço da agricultura de grãos". O uso excessivo de vírgulas e a ausência de espaços entre algumas palavras (como "Amazônia é um bioma importantíssimo para o Brasil e para outros países.Os") também prejudicam a fluidez do texto. Para melhorar, recomenda-se revisar as regras de concordância e pontuação, além de uma reestruturação das frases para evitar confusões.',
   'redacao': 'A Amazônia é um bioma importantíssimo para o Brasil e para outros países.Os motivos para o desmatamento da Amazônia são diverso

### F1-score, Accuracy e QWK

In [10]:
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score, confusion_matrix, ConfusionMatrixDisplay
from collections import Counter

def calculate_qwk_with_margin(all_labels, all_preds, margin=40):
    """
    Calcula o QWK considerando uma margem de tolerância entre o valor real e o valor predito.

    Args:
        all_labels: Lista de rótulos verdadeiros.
        all_preds: Lista de previsões do modelo.
        margin: A margem de tolerância entre o valor real e o valor predito.

    Retorna:
        O QWK médio considerando a margem.
    """
    adjusted_preds = []

    for i in range(len(all_labels)):
        # Verifica se a predição está dentro da margem de tolerância
        if abs(all_preds[i] - all_labels[i]) <= margin:
            adjusted_preds.append(all_labels[i])  # Considera como correto
        else:
            adjusted_preds.append(all_preds[i])  # Usa o valor predito como fallback

    # Calcula o QWK entre rótulos reais e predições ajustadas para a margem
    avg_qwk_with_margin = cohen_kappa_score(all_labels, adjusted_preds, weights='quadratic')
    return avg_qwk_with_margin

# Função para calcular métricas
def calcular_metricas(resultados):
    all_labels = [resultado['nota_real'] for resultado in resultados]
    all_preds = [resultado['nota_predita'] for resultado in resultados]

    # Calcula o F1 Score ponderado
    avg_f1 = f1_score(all_labels, all_preds, average='weighted')

    # Calcula o QWK (Quadratic Weighted Kappa)
    avg_qwk = cohen_kappa_score(all_labels, all_preds, weights='quadratic')

    # calcula qwk com margem
    qwk_with_margin = calculate_qwk_with_margin(all_labels, all_preds, margin=40)

    # Calcula a acurácia
    avg_accuracy = accuracy_score(all_labels, all_preds)

    # Calcula a acurácia considerando a margem de erro de 40 pontos
    acertos_margem = sum(
      1 for true, pred in zip(all_labels, all_preds)
      if abs(true - pred) <= 40  # Checa se a diferença está dentro da margem de 40
    )
    acuracia_margem = acertos_margem / len(all_labels) if len(all_labels) > 0 else 0

    # Retorna as métricas
    return {
        'F1 Score': avg_f1,
        'QWK': avg_qwk,
        'QWK com Margem de 40 pontos': qwk_with_margin,
        'Accuracy': avg_accuracy,
        'Accuracy com Margem de 40 pontos': acuracia_margem,
    }

In [11]:
resultados = extrair_notas_e_feedbacks(loaded_predictions)

In [12]:
def contar_resultados(resultados):
    contagem = {}

    for competencia, itens in resultados.items():
        contagem[competencia] = len(itens)  # Conta quantos itens há em cada lista de competência

    return contagem

# Exemplo de uso
contagem_resultados = contar_resultados(resultados)

# Imprimir contagem
for competencia, quantidade in contagem_resultados.items():
    print(f"{competencia}: {quantidade} resultados")

Competência I: 987 resultados
Competência II: 987 resultados
Competência III: 987 resultados
Competência IV: 987 resultados
Competência V: 987 resultados


In [13]:
for competencia, itens in resultados.items():
  metricas = calcular_metricas(itens)

  # Exibe as métricas calculadas
  print(f"Resultado da {competencia}")
  for nome_metrica, valor in metricas.items():
      print(f"{nome_metrica}: {valor:.4f}")
  print(20*'-')

Resultado da Competência I
F1 Score: 0.4184
QWK: 0.3316
QWK com Margem de 40 pontos: 0.7176
Accuracy: 0.4438
Accuracy com Margem de 40 pontos: 0.9240
--------------------
Resultado da Competência II
F1 Score: 0.4319
QWK: 0.3939
QWK com Margem de 40 pontos: 0.7615
Accuracy: 0.4863
Accuracy com Margem de 40 pontos: 0.9240
--------------------
Resultado da Competência III
F1 Score: 0.3664
QWK: 0.3117
QWK com Margem de 40 pontos: 0.6126
Accuracy: 0.4316
Accuracy com Margem de 40 pontos: 0.8977
--------------------
Resultado da Competência IV
F1 Score: 0.3162
QWK: 0.3344
QWK com Margem de 40 pontos: 0.6395
Accuracy: 0.3860
Accuracy com Margem de 40 pontos: 0.8227
--------------------
Resultado da Competência V
F1 Score: 0.3418
QWK: 0.3952
QWK com Margem de 40 pontos: 0.5847
Accuracy: 0.3718
Accuracy com Margem de 40 pontos: 0.7842
--------------------


### GPT-EVAL